In [1]:
#Python Packages
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb 
import csv
import itertools
from pathlib import Path
from pandas import *
import os.path
import matplotlib.pyplot as plt

In [2]:
#Load in functions from freelymoving ephys
import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/FreelyMovingEphys')
from utils.paths import find
from utils.time import open_time1
from utils.time import open_time
from utils.format_data import open_h5
from utils.format_data import open_ma_h5

#function to get correct frame in case there's lag in the acquisition
def find_first(item, vec):
    return np.argmin(np.abs(vec-item))

In [31]:
##Load Data
# Dictionary of columns in data frame
data_dict = {'date': [],
             'animal': [],
             'task': [],             
             'poke1_ts':[],
             'poke2_ts': [],
             'spout1_ts': [],
             'spout2_ts': [],
             'top1_ts': [],
             'poke1_t0':[],
             'poke2_t0': [],
             'spout1_t0': [],
             'spout2_t0': [],
             'top1_t0': []}
# List of dates for analysis
dates= ['090621','090721','090821','090921','091321']
data_path = Path('~/Desktop/mike_bonsai/recordings/').expanduser()
all_paths = []
#Populate Data Dict with meta data and timestamps   
for date in dates:
    for ani in os.listdir(data_path / date): 
        for task in os.listdir(data_path / date/ ani):
            data_paths = list((data_path / date/ ani/ task).rglob('*.csv'))
            if data_paths != []:
                    data_dict['date'].append(data_paths[1].name.split('_')[0])
                    data_dict['animal'].append(data_paths[1].name.split('_')[1])
                    data_dict['task'].append(data_paths[1].name.split('_')[4])
            for ind, csv in enumerate(data_paths):
                data_dict[data_paths[ind].name.split('_')[5] +'_ts'].append(open_time(csv))
                data_dict[data_paths[ind].name.split('_')[5] +'_t0'].append(open_time(csv)[0])
df = pd.DataFrame(data_dict)


In [32]:
df

,date,animal,task,poke1_ts,poke2_ts,spout1_ts,spout2_ts,top1_ts,poke1_t0,poke2_t0,spout1_t0,spout2_t0,top1_t0
0,090621,G6CK3LT,oa,"[45912.974912, 45935.983936, 45949.62478, 4596...","[45924.182553, 45942.881945, 45955.695244, 459...","[45912.975244, 45914.981017, 45935.984268, 459...","[45924.183027, 45926.196992, 45942.88215, 4594...","[45903.962457, 45903.978739, 45903.995366, 459...",45912.974912,45924.182553,45912.975244,45924.183027,45903.962457
1,090621,G6CK3RT,oa,"[47777.394316, 47843.215001, 47865.924147, 478...","[47823.191488, 47860.340902, 47872.973926, 478...","[47777.394675, 47779.395852, 47843.215347, 478...","[47823.191795, 47825.197606, 47860.341222, 478...","[47774.940876, 47774.955609, 47774.972275, 477...",47777.394316,47823.191488,47777.394675,47823.191795,47774.940876
2,090721,G6CK3LT,oa,"[56501.388723, 56513.120588, 56525.319654, 565...","[56507.422604, 56518.576857, 56529.210982, 565...","[56501.389068, 56502.899648, 56513.120908, 565...","[56507.422937, 56508.927129, 56518.57719, 5652...","[56493.150937, 56493.165504, 56493.18222, 5649...",56501.388723,56507.422604,56501.389068,56507.422937,56493.150937
3,090721,G6CK3RT,oa,"[55078.283648, 55099.309964, 55125.174208, 551...","[55089.913113, 55110.656896, 55130.13888, 5516...","[55078.284147, 55079.795238, 55099.310284, 551...","[55089.913484, 55091.425011, 55110.657408, 551...","[55053.940748, 55053.962022, 55053.973849, 550...",55078.283648,55089.913113,55078.284147,55089.913484,55053.940748
4,090821,G6CK3LT,oa,"[47915.215539, 47939.360576, 47949.779891, 479...","[47920.70446, 47944.274547, 47953.622553, 4796...","[47915.216076, 47916.481715, 47939.360921, 479...","[47920.70478, 47921.961126, 47944.274803, 4794...","[47914.121945, 47914.132441, 47914.149056, 479...",47915.215539,47920.704460,47915.216076,47920.704780,47914.121945
5,090821,G6CK3RT,oa,"[49238.296716, 49262.87863, 49280.415014, 4929...","[49256.168998, 49268.367769, 49285.211584, 492...","[49238.297113, 49239.552768, 49262.87895, 4926...","[49256.169523, 49257.427251, 49268.368166, 492...","[49237.218124, 49237.232934, 49237.2496, 49237...",49238.296716,49256.168998,49238.297113,49256.169523,49237.218124
6,090921,G6CK3LT,oa1,"[53656.865779, 53683.566003, 53705.689996, 537...","[53667.746022, 53691.832371, 53712.183014, 537...","[53656.868006, 53658.398054, 53683.56631, 5368...","[53667.746355, 53669.250342, 53691.832691, 536...","[53655.697152, 53655.701004, 53655.702796, 536...",53656.865779,53667.746022,53656.868006,53667.746355,53655.697152
7,090921,G6CK3RT,oa,"[44348.017216, 44364.283865, 44373.287411, 443...","[44352.035379, 44368.535372, 44379.374604, 443...","[44348.017574, 44349.273907, 44364.284416, 443...","[44352.035609, 44353.295296, 44368.535526, 443...","[44346.792806, 44346.805644, 44346.822259, 443...",44348.017216,44352.035379,44348.017574,44352.035609,44346.792806
8,090921,G6CK3RT,oa1,"[55085.038528, 55117.489638, 55127.54615, 5513...","[55105.536742, 55122.671577, 55132.301977, 551...","[55085.03886, 55086.555264, 55117.489996, 5511...","[55105.537267, 55107.04864, 55122.672064, 5512...","[55083.954521, 55083.969075, 55083.985702, 550...",55085.038528,55105.536742,55085.038860,55105.537267,55083.954521
9,091321,G6CK3LT,oa,"[40616.381606, 40631.734707, 40650.458867, 406...","[40625.012492, 40644.019456, 40655.026278, 406...","[40616.381939, 40617.894835, 40631.73527, 4063...","[40625.012812, 40626.528499, 40644.019814, 406...","[40615.302579, 40615.314419, 40615.331161, 406...",40616.381606,40625.012492,40616.381939,40625.012812,40615.302579


In [33]:
#Align to 
df.poke1_ts = df.poke1_ts - df.top1_t0
df.poke2_ts = df.poke2_ts - df.top1_t0
df.spout1_ts = df.spout1_ts - df.top1_t0
df.spout2_ts = df.spout2_ts - df.top1_t0

In [34]:
#Get all pokes_ts
pokes_ts = []
for ind, row in df.iterrows():
    x = row['poke1_ts'].astype(str).astype(float)
    y = row['poke2_ts'].astype(str).astype(float)
    z = list(itertools.chain(x,y)) 
    z = sorted(z)
    pokes_ts.append(z)
df['pokes_ts'] = pokes_ts

In [35]:
# Number of pokes 
pokes = []
for ind, row in df.iterrows():
    x = row['pokes_ts']
    x=len(x)
    pokes.append(x)
df['poke_num'] = pokes

In [36]:
# Time between pokes in sec
time_between_pokes = []
for ind, row in df.iterrows():
    g = row['pokes_ts']
    g = np.diff(g)
    #g = g[0::2]
    
    time_between_pokes.append(g)
df['time_between_pokes'] = time_between_pokes

In [37]:
#Get 'trials'
## add 
trial_ts = []
for ind, row in df.iterrows():
    x = row['pokes_ts']#.astype(str).astype(float)
    y = [x[i:i+2] for i in range(0, len(x), 2)]
    z =  [x[i+1:i+3] for i in range(0, len(x), 2)]
    t = sorted(y+z)
    trial_ts.append(t)
df['trial_ts']= trial_ts


In [38]:
#Get 'trials #'
trial_num = []
for ind, row in df.iterrows():
    x = row['trial_ts']#.astype(str).astype(float)
    y=len(x)
    trial_num.append(y)
df['trial_num']= trial_num 

In [41]:
time = []
for ind, row in df.iterrows():
    x = row['top1_ts']
    y=len(x)
    time.append(y)
df['time'] = time

In [42]:
df

,date,animal,task,poke1_ts,poke2_ts,spout1_ts,spout2_ts,top1_ts,poke1_t0,poke2_t0,spout1_t0,spout2_t0,top1_t0,pokes_ts,poke_num,time_between_pokes,trial_ts,trial_num,time
0,090621,G6CK3LT,oa,"[9.012454999996407, 32.02147899999545, 45.6623...","[20.220095999997284, 38.919487999999546, 51.73...","[9.012786999999662, 11.018559999996796, 32.021...","[20.22056999999768, 22.234534999995958, 38.919...","[45903.962457, 45903.978739, 45903.995366, 459...",45912.974912,45924.182553,45912.975244,45924.183027,45903.962457,"[9.012454999996407, 20.220095999997284, 32.021...",137,"[11.207641000000876, 11.801382999998168, 6.898...","[[], [9.012454999996407, 20.220095999997284], ...",138,105023
1,090621,G6CK3RT,oa,"[2.4534399999974994, 68.27412499999627, 90.983...","[48.25061199999618, 85.40002600000298, 98.0330...","[2.4537990000026184, 4.454976000000897, 68.274...","[48.25091899999825, 50.25673000000097, 85.4003...","[47774.940876, 47774.955609, 47774.972275, 477...",47777.394316,47823.191488,47777.394675,47823.191795,47774.940876,"[2.4534399999974994, 48.25061199999618, 68.274...",108,"[45.79717199999868, 20.023513000000094, 17.125...","[[2.4534399999974994, 48.25061199999618], [48....",108,59779
2,090721,G6CK3LT,oa,"[8.237785999997868, 19.96965099999943, 32.1687...","[14.271667000000889, 25.425920000001497, 36.06...","[8.238130999998248, 9.748711000000185, 19.9699...","[14.272000000004482, 15.776192000004812, 25.42...","[56493.150937, 56493.165504, 56493.18222, 5649...",56501.388723,56507.422604,56501.389068,56507.422937,56493.150937,"[8.237785999997868, 14.271667000000889, 19.969...",152,"[6.033881000003021, 5.6979839999985415, 5.4562...","[[8.237785999997868, 14.271667000000889], [14....",152,58117
3,090721,G6CK3RT,oa,"[24.342899999996007, 45.36921599999914, 71.233...","[35.972365000001446, 56.71614799999952, 76.198...","[24.34339899999759, 25.854489999997895, 45.369...","[35.972735999996075, 37.484262999998464, 56.71...","[55053.940748, 55053.962022, 55053.973849, 550...",55078.283648,55089.913113,55078.284147,55089.913484,55053.940748,"[24.342899999996007, 35.972365000001446, 45.36...",124,"[11.629465000005439, 9.396850999997696, 11.346...","[[24.342899999996007, 35.972365000001446], [35...",124,75270
4,090821,G6CK3LT,oa,"[1.0935939999981201, 25.23863100000017, 35.657...","[6.582515000001877, 30.152602000001934, 39.500...","[1.0941309999980149, 2.3597700000027544, 25.23...","[6.582835000001069, 7.839181000003009, 30.1528...","[47914.121945, 47914.132441, 47914.149056, 479...",47915.215539,47920.704460,47915.216076,47920.704780,47914.121945,"[1.0935939999981201, 6.582515000001877, 25.238...",177,"[5.4889210000037565, 18.656115999998292, 4.913...","[[], [1.0935939999981201, 6.582515000001877], ...",178,71819
5,090821,G6CK3RT,oa,"[1.0785919999980251, 25.66050600000017, 43.196...","[18.95087400000193, 31.14964499999769, 47.9934...","[1.0789890000014566, 2.3346440000022994, 25.66...","[18.951398999997764, 20.2091270000019, 31.1500...","[49237.218124, 49237.232934, 49237.2496, 49237...",49238.296716,49256.168998,49238.297113,49256.169523,49237.218124,"[1.0785919999980251, 18.95087400000193, 25.660...",185,"[17.872282000003906, 6.709631999998237, 5.4891...","[[], [1.0785919999980251, 18.95087400000193], ...",186,67790
6,090921,G6CK3LT,oa1,"[1.1686269999991055, 27.868850999999268, 49.99...","[12.048869999998715, 36.13521899999614, 56.485...","[1.170853999996325, 2.7009019999968586, 27.869...","[12.049203000002308, 13.553189999998722, 36.13...","[53655.697152, 53655.701004, 53655.702796, 536...",53656.865779,53667.746022,53656.868006,53667.746355,53655.697152,"[1.1686269999991055, 12.048869999998715, 27.86...",175,"[10.88024299999961, 15.819981000000553, 8.2663...","[[], [1.1686269999991055, 12.048869999998715],...",176,78431
7,090921,G6CK3RT,oa,"[1.2244100000025355, 17.49105899999995, 26.494...","[5.242573000003176, 21.742566000000807, 32.581...","[1.22476800000004, 2.4811010000048555, 17.4916...","[5.242803000001004, 6.502489999998943, 21.742